In [1]:
network = "Unet"
encoder = "mobilenet_v2"
pretrain = "imagenet"
inno = 1


In [2]:
    import numpy as np
    import torch
    import segmentation_models_pytorch as smp
    import torchvision
    from torch import nn, optim

In [3]:
    net1 = getattr(smp, network)(
        encoder_name= encoder,        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
        encoder_weights=pretrain,     # use `imagenet` pre-trained weights for encoder initialization
        in_channels=inno,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
        classes=1,                      # model output channels (number of classes in your dataset)
    ) 
   # net.to("cuda")

In [4]:
net1

Unet(
  (encoder): MobileNetV2Encoder(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU6(inplace=True)
          )
          (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (2): InvertedResidual(
        (conv): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(16, 96, kernel_size=(1

In [5]:
# child_counter = 0
# for child in net.children():
#    print(" child", child_counter, "is:")
#    print(child)
#    child_counter += 1

In [6]:
#self.fc2 = nn.Linear(128, 10)

In [7]:
#new_model = nn.Sequential(*list(net.children())[:-1])

In [8]:
#new_model

In [9]:
# class unetver2(nn.Module):
#     def __init__(self):
#         super(unetver2, self).__init__()
#         self.model =  net
#         self.classifier_layer = nn.Sequential(
#             nn.Linear(256*256 , 256*256),
#            # nn.BatchNorm1d(512),
#             nn.Dropout(0.2),
#             nn.Linear(256*256 , 256*256),
#             #nn.Linear(256 ,1)
#         )

In [10]:
# Some imports first
import torch.nn as nn

# New models are defined as classes. Then, when we want to create a model we create an object instantiating this class.
class Unet_FCL(nn.Module):
    def __init__(self,LOAD_VIS_URL=None):
        super(Unet_FCL, self).__init__()
    
         # Start with half the resnet model, swap out the final layer because that's the model we had defined above. 
        model = net1
       # num_final_in = model.fc.in_features
       # model.fc = nn.Linear(num_final_in, 300)
        
        # Now that the architecture is defined same as above, let's load the model we would have trained above. 
       # checkpoint = torch.load(MODEL_PATH)
       # model.load_state_dict(checkpoint)
        
        
#         # Let's freeze the same as above. Same code as above without the print statements
#         child_counter = 0
#         for child in model.children():
#             if child_counter < 6:
#                 for param in child.parameters():
#                     param.requires_grad = False
#             elif child_counter == 6:
#                 children_of_child_counter = 0
#                 for children_of_child in child.children():
#                     if children_of_child_counter < 1:
#                         for param in children_of_child.parameters():
#                             param.requires_grad = False
#                     else:
#                     children_of_child_counter += 1

#             else:
#                 print("child ",child_counter," was not frozen")
#             child_counter += 1
        
        # Now, let's define new layers that we want to add on top. 
        # Basically, these are just objects we define here. The "adding on top" is defined by the forward()
        # function which decides the flow of the input data into the model.
        
        # NOTE - Even the above model needs to be passed to self.
        self.initmodel = nn.Sequential(*list(model.children()))
        self.check_input_shape(x)

        features = self.encoder(x)
        decoder_output = self.decoder(*features)

        masks = self.segmentation_head(decoder_output)

        if self.classification_head is not None:
            labels = self.classification_head(features[-1])
            return masks, labels

        return masks
       # self.projective = nn.Linear(256*256,256*256)
       # self.nonlinearity = nn.ReLU(inplace=True)
       # self.projective2 = nn.Linear(256*256,256*256)
        
    
    # The forward function defines the flow of the input data and thus decides which layer/chunk goes on top of what.
    def forward(self,x):
        x = self.initmodel(x)
      #  x = torch.squeeze(x)
        #x = self.projective(x)
       # x = self.nonlinearity(x)
      #  x = self.projective2(x)
        # x = self.dense
        return x

In [31]:

# Some imports first
import torch.nn as nn

# New models are defined as classes. Then, when we want to create a model we create an object instantiating this class.
class Unet_FCLtest(nn.Module):
    
  def __init__(self):
        super(Unet_FCLtest, self).__init__()
        net1 = nn.Sequential(
            nn.Conv2d(in_channels=1,out_channels=8, kernel_size=5),    # no padding, stride=1, dilation=1 by default
            # Hout = Hin +1 - kernelsize
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # Hout = (Hin -2)/2 + 1
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.Linear(256,256),     # with 32x32 input, the feature map size reduces to 5x5 with 16 channels.
            nn.Linear(256,400),
            nn.Linear(400,64*64), 
           # nn.Linear(128*128,256*256), 
           # nn.Unflatten(1,(8,4,4)),
            nn.Unflatten(1,(1,64,64)),
            nn.Upsample(scale_factor=2, mode='bilinear'),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='bilinear'),
            nn.ReLU(),

            #nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3),

           # nn.Linear(64,256),
           # nn.Linear(256,256*256)
        )
        self.network1 = net1
    
    
  def forward(self,x):
        x = self.network1(x)
    
        return x
    

In [1]:
import pytorch_model_summary as pms
import torch
from customnet1 import customnet1
print(pms.summary(customnet1(), torch.zeros((1, 1, 128, 128)), show_input=False, show_hierarchical=True))


------------------------------------------------------------------------
      Layer (type)         Output Shape         Param #     Tr. Param #
          Conv2d-1     [1, 4, 124, 124]             104             104
            ReLU-2     [1, 4, 124, 124]               0               0
       MaxPool2d-3       [1, 4, 62, 62]               0               0
          Conv2d-4       [1, 4, 58, 58]             404             404
          Conv2d-5       [1, 8, 54, 54]             808             808
            ReLU-6       [1, 8, 54, 54]               0               0
       MaxPool2d-7       [1, 8, 27, 27]               0               0
          Conv2d-8       [1, 8, 23, 23]           1,608           1,608
            ReLU-9       [1, 8, 23, 23]               0               0
         Conv2d-10       [1, 8, 19, 19]           1,608           1,608
           ReLU-11       [1, 8, 19, 19]               0               0
        Flatten-12            [1, 2888]               0        

In [25]:
nettest = Unet_FCLtest()
input = torch.randn(1, 1, 256, 256)
#input.cuda
out = nettest.forward(input)
out.shape

torch.Size([1, 1, 128, 128])

In [ ]:
net1.encoder

In [ ]:
import inspect
print(inspect.getsource(net1.encoder))#.forward))

In [ ]:
import inspect
print(inspect.getsource(net1.forward))

In [ ]:
import inspect
print(inspect.getsource(net2.forward))

In [ ]:
net2 = Unet_FCL()

In [ ]:
net1.forward

In [ ]:
net2.forward

In [ ]:
sum((x - y).abs().sum() for x, y in zip(net1.state_dict().values(), net2.state_dict().values()))


In [ ]:
len(net2.state_dict().values())

In [ ]:
net1.parameters()

In [ ]:
def compare_models(model_1, model_2):
    models_differ = 0
    for key_item_1, key_item_2 in zip(model_1.state_dict().items(), model_2.state_dict().items()):
        if torch.equal(key_item_1[1], key_item_2[1]):
            pass
        else:
            models_differ += 1
            if (key_item_1[0] == key_item_2[0]):
                print('Mismtach found at', key_item_1[0])
            else:
                raise Exception
    if models_differ == 0:
        print('Models match perfectly! :)')

In [ ]:
def compare_models(model_1, model_2):
    models_differ = 0
    for key_item_1, key_item_2 in zip(model_1.state_dict().items(), model_2.state_dict().items()):
        if torch.equal(key_item_1[1], key_item_2[1]):
            pass
        else:
            models_differ += 1
            if (key_item_1[0] == key_item_2[0]):
                print('Mismtach found at', key_item_1[0])
            else:
                raise Exception
    if models_differ == 0:
        print('Models match perfectly! :)')

In [ ]:
compare_models(net1,net2)

In [ ]:
input = torch.randn(1, 1, 256, 256)
#input.cuda
out = net2.forward(input)
out.shape

In [ ]:
out = net1(input)

In [ ]:
out.shape

In [ ]:
input.shape